In [1]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
import numpy as np
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.params import InFlows
from flow.controllers import SimLaneChangeController
from flow.networks import Network
import os
from flow.controllers.routing_controllers import ConstructionRouter
from flow.core.params import SumoLaneChangeParams

from env_constructionV4_padding import myEnv

ADDITIONAL_ENV_PARAMS = {
    "max_accel": 1,
    "max_decel": 1,
}

# time horizon of a single rollout
HORIZON = 1500
# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2

# We place one autonomous vehicle and 22 human-driven vehicles in the network
vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=2)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode="strategic",lcpushy=1.0),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE4","gneE35","gneE38"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "gneE35": ["gneE35","gneE15","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
                "gneE4": ["gneE4","gneE17","gneE11","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
               "gneE8": ["gneE8","gneE9","gneE37","gneE38","gneE39","gneE4.264.110","gneE8"]
               }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="gneE4",
           vehs_per_hour=1000,
            depart_lane="random",
            depart_speed="random",
            color="white")
inflow.add(veh_type="human",
           edge="gneE35",
           vehs_per_hour=5000,
            depart_lane="random",
            depart_speed="random",
            color="blue")


file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/constructionV4.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=False,
        restart_instance=True,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=750,
        clip_actions=False,
        additional_params={
            "target_velocity": 20,
            "sort_vehicles": False,
            "max_accel": 1,
            "max_decel": 1,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config




/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})


2020-03-12 11:11:13,033	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-12 11:11:13,037	INFO resource_spec.py:212 -- Starting Ray with 4.79 GiB memory available for workers and up to 2.41 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-12 11:11:13,640	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_b75e2a10,RUNNING,


(pid=raylet) E0312 11:11:13.926358600    3137 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584025873.926334400","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=raylet) E0312 11:11:13.926807900    3137 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E0312 11:11:13.948817100    3137 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=3153) E0312 11:11:16.148637200    3153 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584025876.148622800","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=3

(pid=3151) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3151)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=3151) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3151)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=3151) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3151)   _np_qint16 = np.dtype([("qint16", np.int16, 1)]

(pid=3153) Warning: Vehicle 'flow_00.10' performs emergency braking with decel=-125.97 wished=4.50 severity=26.99, time=117.20.
(pid=3153) Warning: Vehicle 'flow_00.17' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=119.60.
(pid=3153) Warning: Vehicle 'flow_00.10' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=119.80.
(pid=3153) Warning: Vehicle 'flow_00.17' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=127.20.
(pid=3151) Warning: Vehicle 'flow_10.11' performs emergency braking with decel=-163.48 wished=4.50 severity=35.33, time=128.30.
(pid=3151) Warning: Vehicle 'flow_00.24' performs emergency braking with decel=-9.03 wished=4.50 severity=1.01, time=129.70.
(pid=3153) Warning: Vehicle 'flow_00.17' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=130.30.
(pid=3151) Warning: Vehicle 'flow_00.24' performs emergency braking with decel=-13.37 wished=4.50 severity=1.97, time=1

...

(pid=3153) Warning: Vehicle 'flow_00.24' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=145.40.
(pid=3151) Warning: Vehicle 'flow_00.18' performs emergency braking with decel=-47.40 wished=4.50 severity=9.53, time=161.50.
(pid=3151) Warning: Vehicle 'flow_10.13' performs emergency braking with decel=-15.18 wished=4.50 severity=2.37, time=164.40.
(pid=3151) Warning: Vehicle 'flow_10.13' performs emergency braking with decel=-40.84 wished=4.50 severity=8.08, time=164.50.
(pid=3151) Warning: Vehicle 'flow_10.31' performs emergency braking with decel=-9.03 wished=4.50 severity=1.01, time=168.20.
(pid=3151) Warning: Vehicle 'flow_10.35' performs emergency braking with decel=-51.04 wished=4.50 severity=10.34, time=170.80.
(pid=3153) Warning: Vehicle 'flow_00.24' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=159.60.
(pid=3153) Warning: Vehicle 'flow_10.31' performs emergency braking with decel=-18.13 wished=4.50 severity=3.03, time=16

(pid=3153) Warning: Vehicle 'flow_10.21' performs emergency braking with decel=-52.89 wished=4.50 severity=10.75, time=113.90.
(pid=3151) Warning: Vehicle 'flow_00.22' performs emergency braking with decel=-23.16 wished=4.50 severity=4.15, time=133.70.
(pid=3153) Warning: Vehicle 'flow_00.21' performs emergency braking with decel=-48.26 wished=4.50 severity=9.73, time=117.50.
(pid=3153) Warning: Vehicle 'flow_00.22' performs emergency braking with decel=-9.35 wished=4.50 severity=1.08, time=120.30.
(pid=3151) Warning: Vehicle 'flow_00.38' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=149.30.
(pid=3151) Warning: Vehicle 'flow_00.29' performs emergency braking with decel=-17.21 wished=4.50 severity=2.82, time=149.70.
(pid=3153) Warning: Vehicle 'flow_00.24' performs emergency braking with decel=-11.01 wished=4.50 severity=1.45, time=133.70.
(pid=3151) Warning: Vehicle 'flow_00.14' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=15

(pid=3151) Warning: Vehicle 'flow_10.1' performs emergency braking with decel=-67.57 wished=4.50 severity=14.02, time=113.90.
(pid=3151) Warning: Vehicle 'flow_10.26' performs emergency braking with decel=-12.69 wished=4.50 severity=1.82, time=114.10.
(pid=3151) Warning: Vehicle 'flow_00.20' performs emergency braking with decel=-17.32 wished=4.50 severity=2.85, time=114.10.
(pid=3151) Warning: Vehicle 'flow_10.13' performs emergency braking with decel=-21.48 wished=4.50 severity=3.77, time=114.10.
(pid=3151) Warning: Vehicle 'flow_10.26' performs emergency braking with decel=-12.36 wished=4.50 severity=1.75, time=116.60.
(pid=3153) Warning: Vehicle 'flow_00.12' performs emergency braking with decel=-38.52 wished=4.50 severity=7.56, time=86.20.
(pid=3153) Warning: Vehicle 'flow_10.2' performs emergency braking with decel=-56.75 wished=4.50 severity=11.61, time=91.70.
(pid=3151) Warning: Vehicle 'flow_00.22' performs emergency braking with decel=-14.88 wished=4.50 severity=2.31, time=12

(pid=3151) Warning: Vehicle 'flow_10.55' performs emergency braking with decel=-36.10 wished=4.50 severity=7.02, time=211.40.
(pid=3153) Warning: Vehicle 'flow_10.50' performs emergency braking with decel=-11.13 wished=4.50 severity=1.47, time=200.60.
(pid=3153) Warning: Vehicle 'flow_10.37' performs emergency braking with decel=-51.36 wished=4.50 severity=10.41, time=203.00.
(pid=3153) Warning: Vehicle 'flow_10.56' performs emergency braking with decel=-13.59 wished=4.50 severity=2.02, time=204.40.
(pid=3151) Warning: Vehicle 'flow_10.60' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=216.00.
(pid=3151) Warning: Teleporting vehicle 'flow_10.60'; collision with vehicle 'flow_10.64', lane=':gneJ6_0_2', gap=-1.00, time=216.50 stage=move.
(pid=3151) VALUE ERROR VEL
(pid=3151) VALUE ERROR VEL
(pid=3151) VALUE ERROR VEL
(pid=3151) VALUE ERROR VEL
(pid=3151) VALUE ERROR VEL
(pid=3151) VALUE ERROR VEL
(pid=3151) VALUE ERROR VEL
(pid=3151) Warning: Vehicle 'flow_10

(pid=3151) Warning: Vehicle 'flow_00.22' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=148.10.
(pid=3151) Warning: Vehicle 'flow_00.29' performs emergency braking with decel=-19.45 wished=4.50 severity=3.32, time=150.40.
